### CNN on CIFR Assignment:

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use DropOut layers.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

### Let's import Library

In [1]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras import models,layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization,Activation,Flatten
from tensorflow.keras.optimizers import Adam,SGD

from tqdm import tqdm
import os
from numpy import expand_dims
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler,CSVLogger, Callback,ReduceLROnPlateau

In [2]:
import tensorflow as tf

tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from google.colab import drive
drive.mount('gdrive',force_remount=True)

Mounted at gdrive


In [4]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 250
l = 12
num_filter = 36
compression = 0.5
dropout_rate = 0


In [5]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()
img_height,img_width,channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height)
print(img_width)
print(channel)

170508288/170498071 [==============================] - 11s 0us/step
32
32
3


In [6]:
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [7]:
#convert to one hot encoding
y_train = tf.keras.utils.to_categorical(y_train,num_classes)
y_test = tf.keras.utils.to_categorical(y_test,num_classes)

In [8]:
print(y_train)
print(y_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [9]:
print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


### Model with Dense Block, Transition and Output blocks 

In [10]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp  = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [11]:
num_filter = 36
l = 12
dropout_rate = 0

input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter,(3,3),activation ='relu',use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)
model = Model(inputs=[input], outputs=[output])

model.summary()
          

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 36)   972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 36)   144         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 36)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

**Image Augmentation**

In [12]:
datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=15)

datagen.fit(x_train)

In [13]:
checkpoint = ModelCheckpoint('gdrive/My Drive/cnnoncifar/cifar10_model_save/model-{epoch:03d}-{accuracy:03f}-{val_accuracy:03f}.h5',
                                       monitor='val_accuracy')

csvlog = CSVLogger('gdrive/My Drive/cnnoncifar/csvlog.h5', append = True)

In [14]:
model.compile(loss= "categorical_crossentropy",optimizer=SGD(0.1,momentum = 0.7),metrics=['accuracy'])

In [ ]:

path = 'gdrive/My Drive/cnnoncifar/'
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, 
                    epochs = 30, validation_data =(x_test, y_test), callbacks = [checkpoint,csvlog])
model.save_weights(os.path.join(path, '30epochs.h5'))


Epoch 1/30
390/390 [==============================] - 251s 545ms/step - loss: 1.8326 - accuracy: 0.3547 - val_loss: 1.8010 - val_accuracy: 0.3592
Epoch 2/30
390/390 [==============================] - 207s 530ms/step - loss: 1.4308 - accuracy: 0.4819 - val_loss: 1.3470 - val_accuracy: 0.5154
Epoch 3/30
390/390 [==============================] - 206s 526ms/step - loss: 1.2324 - accuracy: 0.5565 - val_loss: 1.4475 - val_accuracy: 0.5375
Epoch 4/30
390/390 [==============================] - 206s 527ms/step - loss: 1.0704 - accuracy: 0.6174 - val_loss: 1.0906 - val_accuracy: 0.6205
Epoch 5/30
390/390 [==============================] - 206s 527ms/step - loss: 0.9534 - accuracy: 0.6629 - val_loss: 1.1539 - val_accuracy: 0.6232
Epoch 6/30
390/390 [==============================] - 206s 527ms/step - loss: 0.8611 - accuracy: 0.6962 - val_loss: 1.1443 - val_accuracy: 0.6244
Epoch 7/30
390/390 [==============================] - 206s 527ms/step - loss: 0.7931 - accuracy: 0.7219 - val_loss: 0.8913 -

In [ ]:
model.load_weights('/content/gdrive/MyDrive/cnnoncifar/cifar10_model_save/model-030-0.879160-0.840100.h5') 
path = 'gdrive/My Drive/cnnoncifar/'
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, initial_epoch = 30,
                    epochs = 60, validation_data =(x_test, y_test), callbacks = [checkpoint,csvlog])
model.save_weights(os.path.join(path, '60epochs.h5'))

Epoch 31/60
390/390 [==============================] - 205s 523ms/step - loss: 0.3386 - accuracy: 0.8813 - val_loss: 0.5455 - val_accuracy: 0.8327
Epoch 32/60
390/390 [==============================] - 204s 523ms/step - loss: 0.3281 - accuracy: 0.8853 - val_loss: 0.4917 - val_accuracy: 0.8461
Epoch 33/60
390/390 [==============================] - 204s 523ms/step - loss: 0.3218 - accuracy: 0.8858 - val_loss: 0.4398 - val_accuracy: 0.8573
Epoch 34/60
390/390 [==============================] - 204s 523ms/step - loss: 0.3181 - accuracy: 0.8883 - val_loss: 0.5518 - val_accuracy: 0.8365
Epoch 35/60
390/390 [==============================] - 205s 524ms/step - loss: 0.3129 - accuracy: 0.8899 - val_loss: 0.5829 - val_accuracy: 0.8247
Epoch 36/60
390/390 [==============================] - 205s 524ms/step - loss: 0.3045 - accuracy: 0.8934 - val_loss: 0.3996 - val_accuracy: 0.8678
Epoch 37/60
390/390 [==============================] - 205s 524ms/step - loss: 0.3002 - accuracy: 0.8947 - val_loss: 0

In [ ]:
model.load_weights('/content/gdrive/MyDrive/cnnoncifar/cifar10_model_save/model-058-0.925400-0.874700.h5')

path = 'gdrive/My Drive/cnnoncifar/'
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, initial_epoch = 58,
                    epochs = 100, validation_data =(x_test, y_test), callbacks = [checkpoint,csvlog])
model.save_weights(os.path.join(path, '100epochs.h5'))

Epoch 59/100
390/390 [==============================] - 266s 582ms/step - loss: 0.2071 - accuracy: 0.9270 - val_loss: 0.4066 - val_accuracy: 0.8796
Epoch 60/100
390/390 [==============================] - 217s 554ms/step - loss: 0.2085 - accuracy: 0.9263 - val_loss: 0.4008 - val_accuracy: 0.8818
Epoch 61/100
390/390 [==============================] - 217s 554ms/step - loss: 0.2085 - accuracy: 0.9263 - val_loss: 0.3849 - val_accuracy: 0.8863
Epoch 62/100
390/390 [==============================] - 217s 555ms/step - loss: 0.2000 - accuracy: 0.9287 - val_loss: 0.4603 - val_accuracy: 0.8674
Epoch 63/100
390/390 [==============================] - 217s 555ms/step - loss: 0.2026 - accuracy: 0.9285 - val_loss: 0.5404 - val_accuracy: 0.8557
Epoch 64/100
390/390 [==============================] - 217s 555ms/step - loss: 0.2030 - accuracy: 0.9277 - val_loss: 0.3928 - val_accuracy: 0.8810
Epoch 65/100
390/390 [==============================] - 217s 555ms/step - loss: 0.1926 - accuracy: 0.9313 - val_

In [ ]:
model.load_weights('/content/gdrive/MyDrive/cnnoncifar/cifar10_model_save/model-100-0.951640-0.882500.h5')

path = 'gdrive/My Drive/cnnoncifar/'
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, initial_epoch = 100,
                    epochs = 170, validation_data =(x_test, y_test), callbacks = [checkpoint,csvlog])
model.save_weights(os.path.join(path, '170epochs.h5'))

Epoch 101/170
390/390 [==============================] - 259s 566ms/step - loss: 0.1317 - accuracy: 0.9523 - val_loss: 0.3763 - val_accuracy: 0.8964
Epoch 102/170
390/390 [==============================] - 214s 549ms/step - loss: 0.1298 - accuracy: 0.9544 - val_loss: 0.4371 - val_accuracy: 0.8885
Epoch 103/170
390/390 [==============================] - 215s 549ms/step - loss: 0.1334 - accuracy: 0.9531 - val_loss: 0.4418 - val_accuracy: 0.8836
Epoch 104/170
390/390 [==============================] - 214s 548ms/step - loss: 0.1302 - accuracy: 0.9527 - val_loss: 0.4042 - val_accuracy: 0.8973
Epoch 105/170
390/390 [==============================] - 214s 548ms/step - loss: 0.1237 - accuracy: 0.9557 - val_loss: 0.4946 - val_accuracy: 0.8764
Epoch 106/170
390/390 [==============================] - 214s 549ms/step - loss: 0.1227 - accuracy: 0.9561 - val_loss: 0.4145 - val_accuracy: 0.8940
Epoch 107/170
390/390 [==============================] - 214s 549ms/step - loss: 0.1220 - accuracy: 0.9577

In [15]:
model.load_weights('/content/gdrive/MyDrive/cnnoncifar/cifar10_model_save/model-117-0.957860-0.859400.h5')
path = 'gdrive/My Drive/cnnoncifar/'
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, initial_epoch = 117,
                    epochs = 170, validation_data =(x_test, y_test), callbacks = [checkpoint,csvlog])
model.save_weights(os.path.join(path, '170epochs.h5'))

Epoch 118/170
390/390 [==============================] - 261s 571ms/step - loss: 0.1101 - accuracy: 0.9606 - val_loss: 0.4266 - val_accuracy: 0.8918
Epoch 119/170
390/390 [==============================] - 217s 556ms/step - loss: 0.1146 - accuracy: 0.9594 - val_loss: 0.3557 - val_accuracy: 0.9037
Epoch 120/170
390/390 [==============================] - 217s 555ms/step - loss: 0.1103 - accuracy: 0.9610 - val_loss: 0.3594 - val_accuracy: 0.9048
Epoch 121/170
390/390 [==============================] - 217s 556ms/step - loss: 0.1112 - accuracy: 0.9603 - val_loss: 0.4285 - val_accuracy: 0.8923
Epoch 122/170
390/390 [==============================] - 217s 555ms/step - loss: 0.1107 - accuracy: 0.9611 - val_loss: 0.3594 - val_accuracy: 0.9033
Epoch 123/170
390/390 [==============================] - 217s 555ms/step - loss: 0.1059 - accuracy: 0.9628 - val_loss: 0.3718 - val_accuracy: 0.9029
Epoch 124/170
390/390 [==============================] - 217s 554ms/step - loss: 0.1027 - accuracy: 0.9636

In [ ]:
from tensorflow import keras
keras.backend.set_value(model.optimizer.momentum, 0.7)
keras.backend.set_value(model.optimizer.lr, 0.001)
model.load_weights('/content/gdrive/MyDrive/cnnoncifar/cifar10_model_save/model-170-0.973180-0.899800.h5')
path = 'gdrive/My Drive/cnnoncifar/'
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, initial_epoch = 170,
                    epochs = 210, validation_data =(x_test, y_test), callbacks = [checkpoint,csvlog])
model.save_weights(os.path.join(path, '210epochs.h5'))

Epoch 171/210
390/390 [==============================] - 218s 557ms/step - loss: 0.0734 - accuracy: 0.9740 - val_loss: 0.3551 - val_accuracy: 0.9145
Epoch 172/210
390/390 [==============================] - 217s 556ms/step - loss: 0.0645 - accuracy: 0.9769 - val_loss: 0.3490 - val_accuracy: 0.9162
Epoch 173/210
390/390 [==============================] - 218s 558ms/step - loss: 0.0602 - accuracy: 0.9796 - val_loss: 0.3467 - val_accuracy: 0.9177
Epoch 174/210
390/390 [==============================] - 218s 557ms/step - loss: 0.0567 - accuracy: 0.9803 - val_loss: 0.3451 - val_accuracy: 0.9189
Epoch 175/210
390/390 [==============================] - 217s 556ms/step - loss: 0.0569 - accuracy: 0.9806 - val_loss: 0.3409 - val_accuracy: 0.9192
Epoch 176/210
390/390 [==============================] - 217s 554ms/step - loss: 0.0564 - accuracy: 0.9811 - val_loss: 0.3397 - val_accuracy: 0.9196
Epoch 177/210
390/390 [==============================] - 217s 555ms/step - loss: 0.0553 - accuracy: 0.9808

**Observation**

We have used cifar10 dataset and did one hot encoding. As per referenece assignment, used dense block,transition block and output_layer.

To convet on cifar10 dataset, we have used 2D Convolutional neural network and top of that, we have applied dense block to create first block,and for first transition, we have applied transition block on top of first block. Similarly, We created second block,second transition and third block, third transition. On top of last block, we have applied output layer and create the model.

We have used image augmentation technique, and fit train data. We have used categorical_crossentropy as loss function and SGD optimizer with 0.7 momentum and trained the model using metrics "accuracy". 
We have trained by 300 epochs and got test accuracy 90%+.